In [1]:
from fileoperations.fileoperations import getFileNamesInDir
from symbtrdataextractor import symbtrreader, extractor
import os
import json
import pandas as pd 

import pdb

In [2]:
symbtr_cols = ['Sira', 'Kod', 'Nota53', 'NotaAE', 'Koma53', 'KomaAE', 
               'Pay', 'Payda', 'Ms', 'LNS', 'Bas', 'Soz1', 'Offset']
symbtrfolder = os.path.join('..', 'txt')
savefolder = os.path.join('..', '..', 'symbtr_corrections', 'usul_name')

symbtrs = [os.path.splitext(s)[0] for s in getFileNamesInDir('../txt', keyword='*.txt')[2]]

usuldict = json.load(open('./unittests/data/usul_extended.json','r'))
mu2dict = {}
inv_mu2dict = {}
for key, val in usuldict.iteritems():
    for vrt in val['variants']:
        if vrt['mu2_name']:  # if it doesn't have a mu2 name, the usul is not in symbtr collection
            zaman = int(vrt['num_pulses']) if vrt['num_pulses'] else []
            mertebe = int(vrt['mertebe']) if vrt['mertebe'] else []
            mu2dict[vrt['mu2_name']] = {'id':int(vrt['symbtr_internal_id']), 'zaman':zaman,
                                        'mertebe':mertebe}
                                                                                       
            inv_mu2dict[int(vrt['symbtr_internal_id'])] = {'mu2_name':vrt['mu2_name'],
                                                           'zaman':zaman, 'mertebe':mertebe}

In [9]:
for ii, symbTr in enumerate(symbtrs):
    filechanged = False
    symbtrtxtfile = os.path.join(symbtrfolder, symbTr + '.txt')

    df = pd.read_csv(symbtrtxtfile, sep='\t', encoding='utf-8')
    for index, row in df.iterrows():     
        rowchanged = False
        # change null to empty string
        for key, val in row.iteritems():
            if pd.isnull(val):
                row[key] = ''
                rowchanged = True

        if row['Kod'] == 51:
            usul_id = row['LNS']
            usul_name = row['Soz1']
            
            if usul_name:  # name given
                # check if the usul pair matches with the mu2dict
                if mu2dict[usul_name]['id'] == usul_id:
                    if not mu2dict[usul_name]['zaman'] == row['Pay']:
                        print str(ii) + ' ' + symbTr + ', line ' + str(index) + ': ' + usul_name + ' and zaman does not match.'
                        pdb.set_trace()
                    if not mu2dict[usul_name]['mertebe'] == row['Payda']:
                        print str(ii) + ' ' + symbTr + ', line ' + str(index) + ': ' + usul_name + ' and mertebe does not match.'
                        pdb.set_trace()
                else:
                    print str(ii) + ' ' + symbTr + ', line ' + str(index) + ': ' + usul_name + ' and ' + str(usul_id) + ' does not match.'
            elif usul_id:
                if usul_id == -1:
                    print str(ii) + ' ' + symbTr + ', line ' + str(index) + ': Missing usul info'
                else:
                    print str(ii) + ' ' + symbTr + ', line ' + str(index) + ': Filling missing ' + inv_mu2dict[usul_id]['mu2_name']
                    row['Soz1'] = inv_mu2dict[usul_id]['mu2_name']
                    if not inv_mu2dict[usul_id]['zaman'] == row['Pay']:
                        print str(ii) + ' ' + symbTr + ', line ' + str(index) + ': ' + usul_name + ' and zaman does not match.'
                        pdb.set_trace()
                    if not inv_mu2dict[usul_id]['mertebe'] == row['Payda']:
                        print str(ii) + ' ' + symbTr + ', line ' + str(index) + ': ' + usul_name + ' and mertebe does not match.'
                        pdb.set_trace()
                    rowchanged = True
                    filechanged = True
            else:
                pdb.set_trace()

        # reassign
        if rowchanged:
            df.iloc[index] = row
            
    if filechanged:
        savefile = os.path.join(savefolder, symbTr + '.txt')
        df.to_csv(savefile, sep='\t', index=False, encoding='utf-8')


0 beyati--sarki--sofyan--ruhuma_sundugun--sadettin_kaynak, line 183: Missing usul info
11 kurdilihicazkar--sarki--nimsofyan--huzun--selahattin_icli, line 597: Filling missing Semâî
11 kurdilihicazkar--sarki--nimsofyan--huzun--selahattin_icli, line 668: Filling missing Nîmsofyan
15 acemasiran--sazsemaisi--aksaksemai----serafettin_seref_cakar, line 617: Filling missing Aksaksemâî
53 huseyni--sazsemaisi--aksaksemai----lavtaci_andon, line 437: Yürüksemâî and 90 does not match.
53 huseyni--sazsemaisi--aksaksemai----lavtaci_andon, line 553: Filling missing Aksaksemâî
78 tahir--turku--raksan--top_yatagin--aydin, line 234: Filling missing Raksân
86 muhayyer--turku--sofyan--binip_de--denizli, line 235: Filling missing Sofyan
132 irak--sazsemaisi--aksaksemai----serif_muhittin_targan, line 706: Filling missing Aksaksemâî
192 ruyiirak--sazsemaisi--aksaksemai----kani_karaca, line 387: Filling missing Aksaksemâî
198 saba--sazsemaisi--aksaksemai----zaharya, line 287: Yürüksemâî and 87 does not match.

KeyboardInterrupt: 

In [7]:
print (symbTr, ii)


(u'beyati--sarki--sofyan--ruhuma_sundugun--sadettin_kaynak', 209)
